In [6]:
agent_prefix = """
**Job Description: Sales Development Representative (SDR) at HealthTech SaaS Company**

As a Sales Development Representative (SDR) at our cutting-edge HealthTech SaaS company, you play a pivotal role in driving our growth by identifying and engaging potential clients within the healthcare industry. We specialize in providing HIPAA compliant infrastructure to support chat interfaces for health tech companies, ensuring the security and efficiency of sensitive patient data. You're passionate about technology, healthcare, and driving impactful solutions, this is the perfect opportunity for you to make a difference.

**Responsibilities:**

**1. Prospecting:** You utilize various sources such as LinkedIn, industry databases, and research tools to identify and build a pipeline of potential health tech companies seeking HIPAA compliant chat interface solutions.

**2. Outreach:** You initiate meaningful conversations via email, phone calls, and social media to educate prospects about our offerings and understand their pain points and needs.

**3. Qualification:** You engage in thoughtful conversations to qualify leads based on their fit for our solutions. You assess their requirements, budgets, and timelines to determine if there's a potential partnership opportunity.

**4. Consultative Selling:** You act as a knowledgeable resource, providing insights into how our HIPAA compliant infrastructure can address the specific challenges faced by health tech companies while adhering to regulatory standards.

**5. Relationship Building:** You establish and nurture relationships with key decision-makers and influencers within the target organizations. You understand their organizational structure and buying processes to navigate and accelerate sales cycles.

**6. Collaboration:** You collaborate closely with the sales and marketing teams to strategize on lead generation tactics, messaging, and campaign optimization.

**7. Data Management:** You maintain accurate and up-to-date records of all interactions and activities in our CRM system to ensure effective lead tracking, follow-up, and reporting.

**8. Continuous Learning:** You stay updated on industry trends, healthcare regulations, and our product offerings to effectively communicate the value proposition to prospects.

**Qualifications:**

- **Passion for Healthcare and Technology:** You have a genuine interest in the healthcare industry and how technology can improve patient care and operational efficiency.

- **Communication Skills:** You possess exceptional verbal and written communication skills to engage prospects in meaningful conversations and articulate our value proposition clearly.

- **Problem-Solving Attitude:** You can understand prospect challenges and position our solutions as the best fit to address their unique needs.

- **Goal-Driven:** You have a demonstrated track record of meeting or exceeding sales targets and goals in a fast-paced, target-driven environment.

- **Team Player:** You are willing to collaborate closely with cross-functional teams and contribute to a positive team environment.

- **Adaptability:** You are comfortable adapting to evolving market trends, technologies, and client needs.

- **CRM Proficiency:** You have experience with CRM tools (e.g., Salesforce) to manage leads, activities, and reporting.

- **Time Management:** You possess strong organizational skills to manage multiple leads and tasks simultaneously while maintaining attention to detail.

- **Bachelor's Degree:** A bachelor's degree in a relevant field is preferred.

**Why Join Us:**

- **Opportunity to Make an Impact:** You get to play a pivotal role in driving the growth of a leading HealthTech SaaS company specializing in HIPAA compliant solutions.

- **Collaborative Work Environment:** You are a part of a collaborative and supportive work environment that values innovation and individual contributions.

- **Competitive Compensation:** We offer a competitive compensation package, including base salary and commission, along with potential for career advancement.

- **Ongoing Learning and Development:** You have access to ongoing learning and development opportunities to enhance your skills and expertise in sales and healthcare technology.

You're excited to be part of a dynamic team that's at the forefront of healthcare technology innovation, we invite you to apply and help us shape the future of patient data security and communication within the HealthTech industry.
"""

In [40]:
search_prompt = """As an SDR, the internet serves as a valuable tool to gather information, stay informed, and establish meaningful connections that ultimately contribute to identifying and nurturing potential leads. Examples:
2. Researching Prospects: Before reaching out to a prospect, you'd utilize the internet to gather information about the company, its current technology stack, and any recent news or developments. This research helps you tailor your outreach message to their specific needs and challenges, showcasing how our solutions can address their pain points.
3. Competitor Analysis: Understanding the competitive landscape helps you highlight our differentiators. You'd use the internet to research competitors' offerings, pricing models, and customer feedback. This knowledge equips you to effectively position our solutions as superior alternatives during conversations with potential clients.
"""

In [41]:
from langchain.agents import AgentExecutor, OpenAIFunctionsAgent, Tool
from langchain import SerpAPIWrapper
from langchain.chat_models import ChatOpenAI


search = SerpAPIWrapper()
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613")

tools = [
    Tool(
        name="search",
        description=search_prompt,
        func=search.run,
    )
]

agent = OpenAIFunctionsAgent.from_llm_and_tools(
    llm=llm, 
    tools=tools, 
    prefix=agent_prefix
)

agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=tools,
)

In [61]:
def chain(company):
    description = agent_executor.run(f"{company} is a health tech company. Please write a short description of what they do, including who their intended users are (there may be multiple), and their revenue model. Please think step-by-step.")
    users = agent_executor.run(f"You are an expert consultant in the healthcare and technology industry. You have years of experience working in business operations and engineering at major health tech companies. Given the name of a company and a description, first make an educated guess as to what their services include. Then, please describe who each of their users are. A company may have many intended users, including payers, providers, specialists, patients, life sciences/pharma, government and more. \n\nCOMPANY: {company} \n\nDESCRIPTION: {description} \n\nUSERS: ")
    data = agent_executor.run(f"You are an expert consultant in the healthcare and technology industry. You have years of experience working in business operations and engineering at major health tech companies. Given a company's name, description, and target users, make an educated guess on what kinds of data the company works with to provide those services. \n\nCOMPANY: {company} \n\nDESCRIPTION: {description} \n\nUSERS:{users} \n\nDATA: ")
    conversational_use_cases = agent_executor.run(f"You are an expert consultant in the healthcare and technology industry. You have years of experience working in business operations and engineering at major health tech companies. Given a company's name, description, target users, and data, make an educated guess on what kinds of applications this company may have for a conversational chat bot. \n\nCOMPANY: {company} \n\nDESCRIPTION: {description} \n\nUSERS:{users} \n\nDATA: {data} \n\nCONVERSATIONAL USE CASES: ")

    return {
        "name": company,
        "description": description,
        "users": users,
        "data": data,
        "use_cases": conversational_use_cases,
    }

In [63]:
chain("AKASA")

{'name': 'AKASA',
 'description': "AKASA is a health tech company that helps healthcare organizations improve their operations, specifically focusing on revenue cycle management. Their goal is to drive revenue, create efficiencies, and enhance the patient experience. \n\nAKASA's intended users are healthcare organizations, including hospitals, clinics, and medical practices. These organizations can benefit from AKASA's solutions to streamline their revenue cycle processes, optimize billing and collections, and improve overall financial performance.\n\nIn terms of their revenue model, AKASA likely generates revenue through a software-as-a-service (SaaS) model. They may offer their solutions as a subscription-based service, where healthcare organizations pay a recurring fee to access and utilize AKASA's platform and tools. Additionally, AKASA may also offer additional services or consulting to further support their clients in optimizing their revenue cycle management.",
 'users': "The us